In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import traceback
import os

In [2]:
# 保管先フォルダの名称
OUTPUT_DIR = 'livedoor_news'
# 今日のニュース（ライブドアニュース）のURL
URL = 'https://news.livedoor.com/straight_news/'
# ニュースID抽出用の正規表現（名前付きグループ）
REG_URL = r'(?P<L1>(https?://[^/]+/))(?P<L2>([^/]+))/(?P<L3>([^/]+))/(?P<L4>([^/]+))/'
# UserAgent
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'

In [3]:
def create_folder():
    if not os.path.exists(OUTPUT_DIR):
        # フォルダが存在しなければ、フォルダを作成
        os.makedirs(OUTPUT_DIR)

In [4]:
def get_links():
    try:
        # HTTPリクエストヘッダーにUser Agentを設定
        req = Request(URL, data=None, headers={'User-Agent': USER_AGENT})
        with urlopen(req) as res:
            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(res.read().decode('euc_jp', 'ignore'), 'html.parser')
            # 今日のニュース部分を検索し、全てを取得
            soupNewsStraight = soup.find('ul', class_='straightList').find_all('li')

            articles = []
            for idx, soupNews in enumerate(soupNewsStraight):
                # 詳細ページURLをHTMLタグの属性から抽出
                url = soupNews.a.get('href')
                # NewsBodyを検索し取得
                soupNewsBody = soupNews.find('div', class_='straightBody')
                # NewsBodyから各種データを抽出
                article = {
                    'url': url,
                    # ニュースIDを詳細ページURLから抽出
                    'id': re.search(REG_URL, url).groupdict()['L4'],
                    # タイトル／サマリ／提供元／公開日時をHTMLタグの本文から抽出
                    'title': soupNewsBody.find('h3', class_='straightTtl').text,
                    'summary': soupNewsBody.find('p', class_='straightSummary').text,
                    'vender': soupNewsBody.find('p', class_='straightVender').text,
                    'datetime': soupNews.a.time.get('datetime')
                }
                articles.append(article)
                #print('%2d: %s' %(idx + 1, soupNewsBody.find('h3', class_='straightTtl').text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            df.to_csv('%s/livedoor_news_straight.csv' %OUTPUT_DIR, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細（スタックトレース）
        print(traceback.format_exc().rstrip())
        print('=====')

In [5]:
create_folder()
get_links()

In [6]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('livedoor_news/livedoor_news_straight.csv', encoding='utf-8')
df.head()

,url,id,title,summary,vender,datetime
0,https://news.livedoor.com/topics/detail/209881...,20988126,インフルエンザ「今季は大流行の恐れ」 日本感染症学会は接種を呼びかけ,昨季はコロナ禍で感染予防対策が徹底され、インフルエンザ患者が激減した,読売新聞オンライン,2021-10-07 11:56:51
1,https://news.livedoor.com/topics/detail/209880...,20988065,前田敦子が離婚した勝地涼との関係を明かす 3人で動物園に行くことも,休日の過ごし方について、勝地も一緒に親子3人で動物園に行ったと告白,スポニチアネックス,2021-10-07 11:54:53
2,https://news.livedoor.com/topics/detail/209879...,20987916,日本大学の理事ら背任容疑で逮捕 付属病院巡る2億円流出の疑いで,付属病院の建て替え計画を巡り、大学側から2億2000万円を流出させた疑い,読売新聞オンライン,2021-10-07 11:27:55
3,https://news.livedoor.com/topics/detail/209877...,20987741,SUBARUがレガシィの新型モデルを発表 7年ぶりの全面改良,2014年以来7年ぶりの全面改良で、運転支援システム・アイサイトXを標準搭載,時事通信社,2021-10-07 11:07:15
4,https://news.livedoor.com/topics/detail/209875...,20987587,ジョブズの死去から10年経過 アップルの「革新力」などに疑問も,Appleの「革新力」に対しては、疑問を投げかける声もある,読売新聞オンライン,2021-10-07 10:34:32
